# Test Easy Exp

In [ ]:
import sys
from copy import deepcopy

In [ ]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # view the path and verify

In [ ]:
import easy_exp

## Configuration loading

In [ ]:
cfg = easy_exp.cfg.load_configuration()

In [ ]:
cfg

In [ ]:
type(cfg)

In [ ]:
type(cfg["app1"]), type(cfg["__exp__"])

In [ ]:
cfg["__exp__"]

In [ ]:
cfg["app1"]["foo_parse2"]

In [ ]:
cfg["app1.foo_parse2"]

In [ ]:
cfg["app3.foo_str"] = "abc"

In [ ]:
cfg["app3.foo_str"],cfg["app3"]["foo_str"]

## Experiment

In [ ]:
cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters

### Experiment ids

In [ ]:
print(easy_exp.exp.generate_random_id())
print(easy_exp.exp.generate_random_id(key_prefix="B-"))
print(easy_exp.exp.generate_random_id(key_len=8))
print(easy_exp.exp.generate_random_id(key_len=4,key_prefix="FS"))

In [ ]:
import string
import numpy as np
poss_chars = len(string.ascii_letters + string.digits) #62
default_k = 16
default_poss_keys = poss_chars ** default_k
print("Default number of possible keys: 10","^",np.log10(float(default_poss_keys)))

### Hashing

In [ ]:
#The only solution, is save each numpy array / pandas DF / else in the configuration as their own experiment, having a certain experiment ID
numpy_key = "app1.foo_parse2"
np_to_save_elsewhere = cfg[numpy_key]
np_exp_id = 123 #easy_exp.exp.generate_random_id() #get numpy object experiment ID somehow
cfg[numpy_key] = np_exp_id #now cfg doesn't have the numpy array anymore

pandas_key = "app1.foo_parse3"
pd_to_save_elsewhere = cfg[pandas_key]
pd_exp_id = 42 #easy_exp.exp.generate_random_id() #get pandas object experiment ID somehow
cfg[pandas_key] = pd_exp_id #now cfg doesn't have the pandas DF anymore

In [ ]:
easy_exp.exp.hash_config(cfg)

In [ ]:
cfg

In [ ]:
#cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters
easy_exp.exp.get_experiment_id(cfg, exp_cfg)

### Save experiments

In [ ]:
cfg = easy_exp.exp.combine_exp_cfg(cfg, exp_cfg)

In [ ]:
cfg["__exp__"]

In [ ]:
easy_exp.exp.save_experiment(cfg)
# Save experiment can be called directly; it will set the experiment id directly

In [ ]:
easy_exp.exp.get_experiment_id(cfg), easy_exp.exp.get_experiment_id(cfg)
# If the cfg exists, we get
# exp_found True
# Always the same experiment_id

In [ ]:
cfg, exp_cfg = easy_exp.exp.separate_exp_cfg(cfg) # Drop __exp__ key cause it doesn't define an experiment's parameters

In [ ]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), easy_exp.exp.get_experiment_id(cfg,exp_cfg) # Even if the cfg is divided, the result is the same

In [ ]:
cfg["app5"] = 5

In [ ]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), easy_exp.exp.get_experiment_id(cfg,exp_cfg) # If the cfg doesn't exist, we get a different experiment_id. Also, exp_found is False

In [ ]:
easy_exp.exp.get_set_experiment_id(cfg,exp_cfg)

In [ ]:
exp_cfg["experiment_id"] # Now the experiment_id is saved inside exp_cfg

In [ ]:
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]
# get_experiment_id is meant to search for the experiment in the file,
# so, if it doesn't find the cfg in the saved configurations,
# It will not set the exp_id inside exp_cfg

In [ ]:
easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]
# In the same way, get_set_experiment_id is meant to search for the experiment in the file,
# so, if it doesn't find the cfg in the saved configurations,
# it will not return the exp_id inside exp_cfg
# and will overwrite it

In [ ]:
prev_cfg = deepcopy(cfg)

In [ ]:
easy_exp.exp.save_experiment(cfg,exp_cfg)

In [ ]:
prev_cfg == cfg # Check if cfg has been restored

In [ ]:
# Now that the experiment is saved, we will always get the same experiment_id
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

In [ ]:
# If we modify cfg so that is new, we get the same results as before
cfg["new_key"] = "kmasdkasm"
cfg["new_key2"] = {"abc":123}
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

In [ ]:
easy_exp.exp.save_experiment(cfg,exp_cfg)

In [ ]:
exp_cfg["__nosave__"] # Check nosave keys

In [ ]:
cfg["app2.foo_str"] = "modified" # Modify a nosave key

In [ ]:
# If a nosave key is modified, the cfg will still match a saved one
easy_exp.exp.get_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"], easy_exp.exp.get_set_experiment_id(cfg,exp_cfg), exp_cfg["experiment_id"]

### Load experiments

In [ ]:
all_exps = easy_exp.exp.get_experiments(**exp_cfg)
all_exps

In [ ]:
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg={"abc":123}) # If not setting check_type, nothing will be checked
all_exps

In [ ]:
# With contain, sub_cfg must be a "subset" of the dict
to_check = {'new_key': 'kmasdkasm','new_key2': {'abc': 123}}
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg = to_check, check_type="contain") # If not setting check_type, nothing will be checked
all_exps

In [ ]:
#with match, sub_cfg must use relative keys
to_check = {'new_key2.abc': 123}
all_exps = easy_exp.exp.get_experiments(**exp_cfg, sub_cfg = to_check, check_type="match")
all_exps

### Sweep parameter

In [ ]:
#If one parameter is an iterable of parameters to sweep, then use sweep. It will:
#1) cycle on every value in the
#2) set the value to the key
#3) return the value
#4) At the end of the cycle, restore the iterable as value to the key
print(cfg['app1.foo_list1'])
for param_value in cfg.sweep('app1.foo_list1'):
    print(param_value,"___",cfg['app1.foo_list1'])
    easy_exp.exp.get_set_experiment_id(cfg,exp_cfg),easy_exp.exp.save_experiment(cfg,exp_cfg)
print(cfg['app1.foo_list1'])

## Sweep additions

In [ ]:
cfg = easy_exp.cfg.load_configuration("config2")

In [ ]:
cfg

In [ ]:
for value1 in cfg.sweep_additions("app2"):
    print(asdsadas)
    for value2 in cfg.sweep_additions("app1"):
        print(value2)
        print(cfg)
        print()

In [ ]:
cfg["app2"]

In [ ]:
cfg

# Yaml loading times

In [ ]:
import yaml, os, time

In [ ]:
for i in range(10):
    with open(os.path.join("../out/exp",str(i)+".yaml"), 'w') as f:
        yaml.safe_dump(cfg["__exp__"],f)

In [ ]:
start_time = time.time()

for cont in range(10001):
    with open(os.path.join("../out/exp",str(cont%10)+".yaml"), 'r') as f:
       cfg2 = yaml.safe_load(f)

       app = cfg==cfg2

    if cont in [1,10,100,1000,10000]:
        end_time = time.time()
        print(cont,"Time/read:",(end_time-start_time)/cont)
        print(cont,"Tot time:",(end_time-start_time))
        print()

## Argparse

In [ ]:
import argparse, numpy as np

In [ ]:
parser = argparse.ArgumentParser(
                    prog = 'ProgramName',
                    description = 'What the program does',
                    epilog = 'Text at the bottom of help')

In [ ]:
parser.add_argument(
    "--foo.foo", #name or flags - Either a name or a list of option strings, e.g. foo or -f, --foo.
    #optional arguments will be identified by the - prefix; remaining arguments will be assumed to be positional
    # action - The basic type of action to be taken when this argument is encountered at the command line.
# nargs - The number of command-line arguments that should be consumed.
# const - A constant value required by some action and nargs selections.
    default = [1,2,3], #np.ones((2,3))# default - The value produced if the argument is absent from the command line and if it is absent from the namespace object.
# type - The type to which the command-line argument should be converted.
# choices - A sequence of the allowable values for the argument.
# required - Whether or not the command-line option may be omitted (optionals only).
# help - A brief description of what the argument does.
# metavar - A name for the argument in usage messages.
# dest - The name of the attribute to be added to the object returned by parse_args().
    )

In [ ]:
args = parser.parse_args(["--foo","1"])

In [ ]:
eval("np.ones((1,2))")